In [35]:
#importo las librerias necesarias

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")

In [36]:
df = pd.read_csv('Datasets/dataset_limpio.csv')

In [37]:
#chequeo que se importaron bien los datos
df.head()

,id,title,genres,overview,tagline,spoken_languages,decade
0,862,Toy Story,"['Animation', 'Comedy', 'Family']","Led by Woody, Andy's toys live happily in his ...",,['English'],1990
1,8844,Jumanji,"['Adventure', 'Fantasy', 'Family']",When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,"['English', 'Français']",1990
2,15602,Grumpier Old Men,"['Romance', 'Comedy']",A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,['English'],1990
3,31357,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']","Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,['English'],1990
4,11862,Father of the Bride Part II,['Comedy'],Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,['English'],1990


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42196 entries, 0 to 42195
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                42196 non-null  object
 1   title             42196 non-null  object
 2   genres            42196 non-null  object
 3   overview          42196 non-null  object
 4   tagline           42196 non-null  object
 5   spoken_languages  42196 non-null  object
 6   decade            42196 non-null  object
dtypes: object(7)
memory usage: 2.3+ MB


In [39]:
df.isnull().sum()

id                  0
title               0
genres              0
overview            0
tagline             0
spoken_languages    0
decade              0
dtype: int64

la idea principal es crear dos matrices, una para 'overview' y 'tagline' calculando la simiitud usando la matriz TF-IDF ya que este metodo pondera la importancia de cada palabra en relacion a todo el texto, lo que captura mejor las palabras claves o terminos especificos. 
para 'genres', 'decade' y 'spoken_languages' usar una matriz de conteo ya que esta simplemente cuenta la frecuencia de cada palabra sin tener en cuenta el resto del texto, considerando que estos datos son más cercanos a un metadato que a datos de lenguaje como si ocurre en 'overview' y 'tagline'

vamos a preparar los datos para crear las matrices

In [40]:
#eliminación de caracteres especiales y conversión a minúsculas:
df['overview'] = df['overview'].str.replace('[^\w\s]', '').str.lower().str.replace(",", "").str.replace(".", "").str.replace("!", "")
df['tagline'] = df['tagline'].str.replace("[^\w\s]", "").str.lower().str.replace(",", "").str.replace(".", "").str.replace("!", "")

In [41]:
#eliminación de palabras vacías (stopwords) y lematización:
nltk.download('stopwords')
nltk.download('wordnet')

#nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

df['overview'] = df['overview'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stop_words]))
df['tagline'] = df['tagline'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stop_words]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ssanjua/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ssanjua/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
df['overview'].iloc[0]

"led woody andy's toy live happily room andy's birthday brings buzz lightyear onto scene afraid losing place andy's heart woody plot buzz circumstance separate buzz woody owner duo eventually learns put aside difference"

In [43]:
df['tagline'].iloc[1]

'roll dice unleash excitement'

In [44]:
#extraccion de overview + tagline
text_data = df[['overview', 'tagline']]

#combinamos los campos en un solo texto
df['texto_combinado'] = text_data['overview'] + ' ' + text_data['tagline']

In [45]:
#eliminamos las columnas ya que no las necesitaremos mas
df = df.drop('overview', axis=1)
df = df.drop('tagline', axis=1)

In [46]:
df.head()

,id,title,genres,spoken_languages,decade,texto_combinado
0,862,Toy Story,"['Animation', 'Comedy', 'Family']",['English'],1990,led woody andy's toy live happily room andy's ...
1,8844,Jumanji,"['Adventure', 'Fantasy', 'Family']","['English', 'Français']",1990,sibling judy peter discover enchanted board ga...
2,15602,Grumpier Old Men,"['Romance', 'Comedy']",['English'],1990,family wedding reignites ancient feud next-doo...
3,31357,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",['English'],1990,cheated mistreated stepped woman holding breat...
4,11862,Father of the Bride Part II,['Comedy'],['English'],1990,george bank recovered daughter's wedding recei...


In [47]:
#quito los espacios en los generos formados por mas de una silaba
df['genres'] = df['genres'].str.replace(" ", "")

In [48]:
#duplico 'genres' para darle mas importancia y las uno en un campo
df['combined_features'] = df['genres'] + df['genres'] + df['decade'] + df['spoken_languages']
df['combined_features'][7]

"['Action','Adventure','Drama','Family']['Action','Adventure','Drama','Family']1990['English', 'Deutsch']"

In [49]:
#elimino las columnas que ya no me sirven
df = df.drop('genres', axis=1)
df = df.drop('decade', axis=1)
df = df.drop('id', axis=1)
df = df.drop('spoken_languages', axis=1)

In [50]:
#aplico algunos espacios mas para asegurarme que queden todas las palabras separadas
df['combined_features'] = df['combined_features'].str.replace("'", " ").str.replace("[", " ").str.replace("]", " ").str.replace(",", " ").str.lower()

In [51]:
df.head(5)

,title,texto_combinado,combined_features
0,Toy Story,led woody andy's toy live happily room andy's ...,animation comedy family animation c...
1,Jumanji,sibling judy peter discover enchanted board ga...,adventure fantasy family adventure ...
2,Grumpier Old Men,family wedding reignites ancient feud next-doo...,romance comedy romance comedy 1990 ...
3,Waiting to Exhale,cheated mistreated stepped woman holding breat...,comedy drama romance comedy drama ...
4,Father of the Bride Part II,george bank recovered daughter's wedding recei...,comedy comedy 1990 english


In [52]:
#exporto como nuevo dataset para usarlo en mi modelo de ML en la API
df.to_csv('Datasets/dataset_ML.csv', index=False)

creo la matriz TF-IDF con 'texto_combinado' = 'overview' + 'tagline'

In [53]:
# Crear el vectorizador TF-IDF. lo limito a 50000 para que no sea tan grande
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', max_features=50000)

# Aplicar el vectorizador a los datos de texto
tfidf_matrix = vectorizer.fit_transform(df['texto_combinado'])

In [54]:
tfidf_matrix.shape

(42196, 50000)

In [55]:
#similaritud del coseno: 
coseno_sim_text = cosine_similarity(tfidf_matrix)

#toma entre 35s a 50s

ahora creamos la matriz de conteo con 'combined_features' = 'genres' + 'decade' + 'spoken_languages'

In [56]:
#Vectorización de texto
vectorizer = CountVectorizer(stop_words='english')
feature_matrix = vectorizer.fit_transform(df['combined_features'])

In [57]:
feature_matrix.shape

(42196, 246)

In [58]:
#similaritud del coseno para esta matriz
coseno_sim_features = cosine_similarity(feature_matrix)

#tarda entre 1min y 2min

In [59]:
#combina las similitudes utilizando suma ponderada, donde le doy más peso a el tema de la película en relación
#a el lenguaje y la década
combined_similarity = 0.6 * coseno_sim_text + 0.4 * coseno_sim_features

#tarda mas 4min

In [60]:
#creamos una función para nuestro sistema de recomendación
def get_recommendations(title):
    titles = df['title']
    indices = pd.Series(df.index, index=df['title'])  
    idx = indices[title]
    sim_scores = list(enumerate(combined_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    
    return df['title'].iloc[movie_indices]

In [61]:
get_recommendations('Toy Story')

2961              Toy Story 2
14764             Toy Story 3
23200               Small Fry
20865    Toy Story of Terror!
726             A Close Shave
Name: title, dtype: object

In [62]:
get_recommendations('Minions')

20145    Despicable Me 2
5271     Stuart Little 2
40892    Despicable Me 3
18828     Wreck-It Ralph
41898             Banana
Name: title, dtype: object

In [63]:
get_recommendations('Avatar')

6320     Lara Croft Tomb Raider: The Cradle of Life
25040                                Thor: Ragnarok
9904                                 Fantastic Four
14579                                  Solomon Kane
2502                                    Superman II
Name: title, dtype: object

In [64]:
get_recommendations('Titanic')

2045      Return to Paradise
342              Bitter Moon
652                     Fear
34752    Allyson Is Watching
14595                Voyager
Name: title, dtype: object

In [65]:
get_recommendations('Superman')

2502                          Superman II
10733                    Superman Returns
30221                Atom Man vs Superman
2504     Superman IV: The Quest for Peace
20058                        Man of Steel
Name: title, dtype: object

In [66]:
df.shape

(42196, 3)

el modelo funciona bastante bien hasta acá, pero demora demasiado en ejecutarse por lo que desde ya voy a abandonar la idea de aplicar un filtro para que considere las puntuaciones y la popularidad porque va a seguir sumando carga al modelo.

para poder deployar y hacerlo funcionar en tiempo razonable voy a tener que reducir el tamaño de mi set de prueba, intentando seguir con las dos matrices pero entendiendo que lo que más tiempo le lleva al modelo es encontrarla similaritud entre ambas.
ahora voy a reducir el dataset para corroborar si los tiempos de procesamiento mejoran.

In [80]:
#creo un nuevo dataset mas pequeño
sm_df = df.sample(n=8000, random_state=42)

#le reseteo el indice para evitar problemas
sm_df=sm_df.reset_index(drop=True)

In [81]:
sm_df.head()

,title,texto_combinado,combined_features
0,The Business of Fancydancing,seymour polatkin successful gay indian poet sp...,music drama music drama 2000 english
1,Dollman,brick bardo traveller outer space forced land ...,action comedy crime sciencefiction ...
2,The Dukes,dukesa doo wop group top world 17 struggling s...,comedy crime drama music comedy c...
3,Dragon Wars: D-War,based korean legend unknown creature return de...,fantasy drama horror action thriller...
4,Tom and Huck,mischievous young boy tom sawyer witness murde...,action adventure drama family actio...


In [82]:
# Crear el vectorizador TF-IDF. lo limito a 50000 para que no sea tan grande
vectorizerSML = TfidfVectorizer(analyzer='word', stop_words='english', max_features=50000)

# Aplicar el vectorizador a los datos de texto
tfidf_matrixSML = vectorizerSML.fit_transform(sm_df['texto_combinado'])

tfidf_matrixSML.shape

(8000, 30238)

In [83]:
#similaritud del coseno del TF-IDF: 
coseno_sim_textdSML = cosine_similarity(tfidf_matrixSML)

#demora menos de 1s

In [84]:
#conteo de vectores con los features
vectorizerSML_features = CountVectorizer(stop_words='english')
feature_matrixSML = vectorizerSML_features.fit_transform(sm_df['combined_features'])

feature_matrixSML.shape

(8000, 167)

In [85]:
#similaritud del coseno
coseno_sim_featuresSML = cosine_similarity(feature_matrixSML)

#demora menos de 1s

In [86]:
# Combina las similitudes utilizando algún método (por ejemplo, suma ponderada)
combined_similaritySML = 0.6 * coseno_sim_textdSML + 0.4 * coseno_sim_featuresSML

#demora menos de 1s

In [87]:
def get_recommendationsSML(title):
    titles = sm_df['title']
    indices = pd.Series(sm_df.index, index=sm_df['title'])  
    if title not in indices:
        return "La película no está en el dataset reducido"
    idx = indices[title]
    sim_scores = list(enumerate(combined_similaritySML[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Obtenemos las 5 películas más similares
    movie_indices = [i[0] for i in sim_scores]
    
    return df['title'].iloc[movie_indices]

In [88]:
get_recommendationsSML('The Dukes')

6988               Love at Large
563            Little Big League
7384    Smiles of a Summer Night
7141                     Godsend
2982        The James Dean Story
Name: title, dtype: object

In [89]:
get_recommendationsSML('Toy Story')

'La película no está en el dataset reducido'

In [90]:
#definicion final para API:
def recomendacion(title:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    indices = sm_df[sm_df['title'] == title]
    if indices.empty:
        return "La película no está en el dataset reducido"

    idx = indices.index[0]
    vectorizerTF = TfidfVectorizer(analyzer='word', stop_words='english')
    tfidf_matrixTF = vectorizerTF.fit_transform(sm_df['texto_combinado'])
    coseno_sim_text = cosine_similarity(tfidf_matrixTF)
    vectorizer_features = CountVectorizer(stop_words='english')
    feature_matrix = vectorizer_features.fit_transform(sm_df['combined_features'])
    coseno_sim_features = cosine_similarity(feature_matrix)
    combined_similarity = 0.6 * coseno_sim_text + 0.4 * coseno_sim_features
    sim_scores = list(enumerate(combined_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Obtenemos las 5 películas más similares
    movie_indices = [i[0] for i in sim_scores]
    movie_titles = [sm_df['title'].iloc[i].title() for i in movie_indices]

    return {'lista recomendada': movie_titles} 

In [92]:
recomendacion('The Dukes')

{'lista recomendada': ["Mini'S First Time",
  'Girltrash: All Night Long',
  'Taking Woodstock',
  'Duets',
  'Little Fish, Strange Pond']}

In [93]:
recomendacion('Toy Story')

'La película no está en el dataset reducido'

In [94]:
recomendacion('Dollman')

{'lista recomendada': ['Evil Alien Conquerors',
  'Crossworlds',
  'Space Cop',
  'The Hard Way',
  'Darkman']}